In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OrdinalEncoder,OneHotEncoder

In [2]:
train_mean = pd.read_csv("datasets/mean_data_train.csv")
train_median = pd.read_csv("datasets/median_data_train.csv")
test_mean = pd.read_csv("datasets/mean_data_test.csv")
test_median = pd.read_csv("datasets/median_data_test.csv")

In [3]:
tarihler = train_mean.tarih

In [4]:
train_mean.drop(["Unnamed: 0","tarih"],axis=1,inplace=True)
test_mean.drop(["Unnamed: 0","tarih"],axis=1,inplace=True)
train_median.drop(["Unnamed: 0","tarih"],axis=1,inplace=True)
test_median.drop(["Unnamed: 0","tarih"],axis=1,inplace=True)

In [5]:
train_mean["ilce"] = pd.Categorical(train_mean["ilce"])
train_median["ilce"] = pd.Categorical(train_mean["ilce"])

In [6]:
X_mean = train_mean.drop("bildirimsiz_sum",axis=1)
X_median = train_median.drop("bildirimsiz_sum",axis=1)
y_mean = train_mean["bildirimsiz_sum"]
y_median = train_median["bildirimsiz_sum"]

In [7]:
ord_coder = OrdinalEncoder()
ohe = OneHotEncoder()
sc = StandardScaler()
mm = MinMaxScaler()

## Mean

In [8]:
dummy_mean = ohe.fit_transform(X_mean[["ilce"]])
dummy_mean = dummy_mean.toarray()
features = ohe.get_feature_names_out(["ilce"])
dummies_mean = pd.DataFrame(data=dummy_mean,columns=features)
X_mean = X_mean.drop("ilce",axis=1)
X_mean = pd.concat([X_mean,dummies_mean],axis=1)

In [ ]:
X_mean["ilce"] = ord_coder.fit_transform(X_mean[["ilce"]])

In [ ]:
cols = dummies_mean.columns.to_list()
cols.append("bildirimli_sum")
mm_columns = X_mean.drop(cols,axis=1).columns

In [ ]:
mm_columns = X_mean.drop(["ilce","bildirimli_sum"],axis=1).columns

In [ ]:
mm.fit(X_mean[mm_columns])
X_mean[mm_columns] = mm.fit_transform(X_mean[mm_columns])

In [ ]:
for col in X_mean.columns:
    X_mean[col] = X_mean[col].astype("float64")

## Median

In [8]:
dummy_mean = ohe.fit_transform(X_median[["ilce"]])
dummy_mean = dummy_mean.toarray()
features = ohe.get_feature_names_out(["ilce"])
dummies_median = pd.DataFrame(data=dummy_mean,columns=features)
X_median = X_median.drop("ilce",axis=1)
X_median = pd.concat([X_median,dummies_median],axis=1)

In [10]:
X_median["ilce"] = ord_coder.fit_transform(X_median[["ilce"]])

KeyError: "None of [Index(['ilce'], dtype='object')] are in the [columns]"

In [9]:
cols = dummies_median.columns.to_list()
cols.append("bildirimli_sum")
mm_columns = X_median.drop(cols,axis=1).columns

In [19]:
mm_columns = X_median.drop(["ilce","bildirimli_sum"],axis=1).columns

KeyError: "['ilce'] not found in axis"

In [10]:
mm.fit(X_median[mm_columns])
X_median[mm_columns] = mm.fit_transform(X_median[mm_columns])

In [11]:
for col in X_median.columns:
    X_median[col] = X_median[col].astype("float64")

## Split Dataset

In [15]:
def create_sequence(X, y, step=1):
    X_seq, y_seq = [], []
    for i in range(len(X) - step):
        X_seq.append(X.iloc[i:i+step])
        y_seq.append(y.iloc[i+step])
    return np.array(X_seq), np.array(y_seq)

In [16]:
X_seq,y_seq = create_sequence(X_median,y_median,10)

## Prepare Test Set

In [12]:
dummy_median = ohe.transform(test_median[["ilce"]])
dummy_median = dummy_median.toarray()
features = ohe.get_feature_names_out(["ilce"])
dummies_median = pd.DataFrame(data=dummy_median,columns=features)
test_median = test_median.drop("ilce",axis=1)
test_median = pd.concat([test_median,dummies_median],axis=1)

In [13]:
mm.fit(test_median[mm_columns])
test_median[mm_columns] = mm.fit_transform(test_median[mm_columns])

In [14]:
def test(test,step=5):
    X_seq = [] 
    for i in range(len(test) - step):
        X_seq.append(test.iloc[i:i+step])
    return np.array(X_seq)

In [17]:
test_seq = test(test_median,10)

In [18]:
test_seq.shape

(1306, 10, 58)

## DNN

In [19]:
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(2000,"tanh"))
nn.add(tf.keras.layers.Dense(4000,"tanh"))
nn.add(tf.keras.layers.Dense(1000,"tanh"))
nn.add(tf.keras.layers.Dense(500,))
nn.add(tf.keras.layers.Dense(250,"tanh"))
nn.add(tf.keras.layers.Dense(50,"tanh"))
nn.add(tf.keras.layers.Dense(1,"tanh"))
nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss="mae")

In [ ]:
nn.fit(X_mean,y_mean,epochs=150,)

## LSTM

In [20]:
model = tf.keras.models.Sequential()

In [21]:
model.add(tf.keras.layers.LSTM(50, activation="tanh",return_sequences=True,input_shape=(X_seq.shape[1],X_seq.shape[2])))
model.add(tf.keras.layers.LSTM(20,activation="tanh"))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss="mae")

In [22]:
model.fit(X_seq,y_seq,epochs=500,batch_size=32)

Epoch 1/500
1613/1613 [==============================] - 23s 7ms/step - loss: 2.9036
Epoch 2/500
1613/1613 [==============================] - 11s 7ms/step - loss: 2.7817
Epoch 3/500
1613/1613 [==============================] - 11s 7ms/step - loss: 2.7610
Epoch 4/500
1613/1613 [==============================] - 12s 7ms/step - loss: 2.7478
Epoch 5/500
1613/1613 [==============================] - 11s 7ms/step - loss: 2.7350
Epoch 6/500
1613/1613 [==============================] - 11s 7ms/step - loss: 2.7252
Epoch 7/500
1613/1613 [==============================] - 12s 7ms/step - loss: 2.7131
Epoch 8/500
1613/1613 [==============================] - 12s 7ms/step - loss: 2.7028
Epoch 9/500
1613/1613 [==============================] - 12s 7ms/step - loss: 2.6917
Epoch 10/500
1613/1613 [==============================] - 12s 8ms/step - loss: 2.6790
Epoch 11/500
1613/1613 [==============================] - 13s 8ms/step - loss: 2.6691
Epoch 12/500
1613/1613 [==============================] - 13s 8

In [23]:
model.evaluate(X_seq,y_seq)

1613/1613 [==============================] - 11s 5ms/step - loss: 1.3300


1.3300468921661377

In [24]:
def predict_future(X, time_steps):
    future_predictions = []
    current_batch = X[-1].reshape(1, time_steps, X.shape[2])
    for i in range(1316):
        future_pred = model.predict(current_batch)
        future_predictions.append(future_pred)
        current_batch = np.roll(current_batch, -1, axis=1)
        current_batch[0, -1, 0] = future_pred 
    return future_predictions

In [25]:
preds = predict_future(test_seq,10)

1/1 [==============================] - 0s 26ms/step


C:\Users\LOKMAN\AppData\Local\Temp\ipykernel_10732\1807619653.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_batch[0, -1, 0] = future_pred


1/1 [==============================] - 0s 22ms/step


In [28]:
preds = np.array(preds).flatten()

In [32]:
tf.keras.losses.mean_absolute_error(preds,pd.read_csv("SVR/submission_SVR.csv").iloc[:,1])

<tf.Tensor: shape=(), dtype=float64, numpy=3.4979097370028587>

In [ ]:
inverse_ilceler = ohe.inverse_transform(test_mean[dummies_mean.columns])

In [ ]:
inverse_ilceler = inverse_ilceler.reshape((1316,))

In [ ]:
test1["ilce"] = inverse_ilceler
test1["tarih"] = test1_dates

In [ ]:
test1["tarih"] = test1["tarih"].astype("str")

In [ ]:
unique_id = test1["tarih"] + "-" + test1["ilce"]